# 📓 Custom App using DummyAPI

The files in this folder code a custom app that resembles an LLM app with
seperate classes for common app components. None of these components make
network calls but pretend to otherwise operate like real components.

__WARNING__: This example must be run from the git repository as it imports
files that are not included in a PIP distribution.

In [ ]:
from pathlib import Path
import sys

# If using trulens from the repository, add the parent directory to the path:
sys.path.append(str(Path().cwd().parent.parent.parent.parent.resolve()))

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp

from trulens_eval import Tru
from trulens_eval.tru_custom_app import TruCustomApp

tru = Tru()
tru.reset_database()

In [ ]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
# Show instrumented components and methods.

ta.print_instrumented()

In [ ]:
# Normal usage (without tracing):
ca.respond_to_query(query="What is the capital of Indonesia?")

# Instrumented usage:
with ta as recorder:
    response = ca.respond_to_query(query="What is the capital of Indonesia?")

record = recorder.get()

In [ ]:
# Show the app output:

# response

In [ ]:
# Show the instrumentation record.

# record.model_dump()

In [ ]:
# Start the dasshboard. If you running from github repo, include the _dev arg below:
tru.start_dashboard(
    _dev=Path().cwd().parent.parent.parent.parent.resolve(),
    force=True
)

# Custom App options

The custom app used above has some configuration options or call variants with
differing behaviours:

- Simulated delay and memory usage controlled by `Dummy` class `delay` and
  `alloc` attributes. All other components subtype `Dummy` hence also implement
  simulated delays and allocations.

- Non-determinism control via the `Dummy` class `seed` attribute. Randomness in
  some components can be somewhat controlled by this attribute. The control is
  presently incomplete as threading and async usage (described below) is not
  controlled by `seed`.

- Async methods. These begin with the letter "a" followed by the sync method
  name.

- Use of parallelism for processing retrieved chunks. This uses either threads
  or async depending on which method variant is called. This is controlled by
  the `CustomApp` `use_parallel` flag.

- Nested use of custom app inside a custom app component. This is controlled by
  the `CustomAgent` `use_app` flag.

- Use of trulens_eval to record the invocation of the above nested model. This
  is controlled by the `CustomAgent` `use_recorder` flag.



In [ ]:
# Normal usage (without tracing):
await ca.arespond_to_query(query="What is the capital of Indonesia?")

# Instrumented usage:
with ta as recorder:
    response = await ca.arespond_to_query(query="What is the capital of Indonesia?")

record = recorder.get()